In [1]:
import glob
from sort import *
import numpy as np

/home/jvwoerden/.local/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
video_paths = glob.glob('../data/Fudan/*/*')

In [3]:
import json
# Get all the frames of a single video (all frames in given directory)
def get_all_frames(path):
    files = glob.glob('{}/*.json'.format(path))
    files.sort()
    ret = []
    for file in files:
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Fix weird json syntax in file
        data = data[list(data.keys())[0]]
        t = {}
        t['filename'] = '{}/{}'.format(path, data['filename'])
        t['centers'] = []
        t['bboxes'] = []
        
        for region in data['regions']:
            
            t['bboxes'].append([
                region['shape_attributes']['x'],
                region['shape_attributes']['y'],
                region['shape_attributes']['x'] + region['shape_attributes']['width'],
                region['shape_attributes']['y'] + region['shape_attributes']['height'],
            ])
            
            t['centers'].append({
                'x': region['shape_attributes']['x'] + region['shape_attributes']['width']/2,
                'y': region['shape_attributes']['y'] + region['shape_attributes']['height']/2,
            })
            
        ret.append(t)
    return ret

In [4]:
from PIL import ImageDraw, Image, ImageFont

def get_img_with_centers(frame, track_id = False):
    img = Image.open(frame['filename'])
    draw = ImageDraw.Draw(img)

    for i, region in enumerate(frame['centers']):
        r = 4
        leftUpPoint = (region['x']-r, region['y']-r)
        rightDownPoint = (region['x']+r, region['y']+r)
        twoPointList = [leftUpPoint, rightDownPoint]
        draw.ellipse(twoPointList, fill=(255,0,0,255))
        if track_id:
            leftUpPoint = (region['x']-r+8, region['y']-r+8)
            draw.text(leftUpPoint, str(int(frame['tracked_id'][i])))
        
    return img

In [5]:
import copy

def get_track_frames(frames):
    frames = copy.deepcopy(frames)
    mot_tracker = Sort()
    for i, frame_info in enumerate(frames):
        original_bboxes = np.array(frame_info['bboxes'])
        
        track_bbs_ids = mot_tracker.update(original_bboxes)
        frames[i]['tracked_id'] = [int(i) for i in track_bbs_ids[:,-1].tolist()]
        
        bboxes = []
        centers = []
        
        for item in track_bbs_ids:
            listed_item = item.tolist()
            bboxes.append(listed_item)
            centers.append({
                'x': (listed_item[0] + listed_item[2]) / 2,
                'y': (listed_item[1] + listed_item[3]) / 2
            })
            
        frames[i]['bboxes'] = bboxes
        frames[i]['centers'] = centers
    return frames

In [9]:
frames = get_all_frames(video_paths[2])
frames = get_track_frames(frames)
print(frames[0])

/home/jvwoerden/.local/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


{'filename': '../data/Fudan/test_data/69/001.jpg', 'centers': [{'x': 1608.5, 'y': 145.0}, {'x': 1626.0, 'y': 97.5}, {'x': 1627.0, 'y': 76.5}, {'x': 1263.0, 'y': 370.5}, {'x': 470.5, 'y': 304.0}, {'x': 488.0, 'y': 303.5}, {'x': 1550.0, 'y': 17.0}, {'x': 107.5, 'y': 829.0}, {'x': 756.5, 'y': 123.0}, {'x': 1043.5, 'y': 80.5}, {'x': 599.0, 'y': 15.5}, {'x': 550.0, 'y': 56.0}, {'x': 23.0, 'y': 537.0}, {'x': 531.5, 'y': 58.5}, {'x': 924.0, 'y': 888.0}, {'x': 769.0, 'y': 68.5}, {'x': 304.0, 'y': 519.5}, {'x': 728.0, 'y': 145.5}, {'x': 169.5, 'y': 26.5}, {'x': 589.5, 'y': 492.5}, {'x': 582.0, 'y': 520.5}, {'x': 72.0, 'y': 317.5}, {'x': 1476.0, 'y': 983.5}, {'x': 1341.5, 'y': 97.5}, {'x': 1228.5, 'y': 97.0}, {'x': 239.5, 'y': 43.0}, {'x': 153.0, 'y': 797.5}, {'x': 671.5, 'y': 809.0}, {'x': 481.5, 'y': 204.5}, {'x': 962.0, 'y': 957.5}, {'x': 358.5, 'y': 474.5}, {'x': 1817.5, 'y': 280.5}, {'x': 908.5, 'y': 68.0}, {'x': 1281.5, 'y': 152.5}, {'x': 857.5, 'y': 922.0}, {'x': 1080.0, 'y': 266.0}, {'x'

In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.animation as animation
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from IPython.display import clear_output
def generate_video(frame_info):
    
    c_frames = len(frame_info)

    fig,ax = plt.subplots(1, figsize=(14,8))
    fig.tight_layout()
    plt.axis('off')

    pbar = tqdm(total=c_frames)
    pbar.set_description("Generating frames")
    imgs = []
    for frame in frame_info[0:c_frames]:
        img = get_img_with_centers(frame, track_id = True)

        imgs.append([ax.imshow(img)])
        pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Stitching frames together")
    ani = animation.ArtistAnimation(fig, imgs, interval=40, blit=True)
    pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Prepare for display")
    tag = ani.to_html5_video()
    pbar.update(1)
    pbar.close()
    plt.close()

    clear_output()
    display(HTML(tag))

In [8]:
generate_video(frames)